In [1]:
import numpy as np
import matplotlib.pyplot  as plt
import time

In [2]:
# set a randome function to generate the random number
def random_number (seed):
    # set the seed
    seed = hash(seed)
    # set the parameters
    a = 1664525
    c = 1013904223
    m = 2**32
    # generate the random number
    seed = (a * seed + c) % m
    return seed

In [3]:
# softmax function
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [4]:
def P_WM (lat, gamma, beta, delta, pre_s):
    # compute the hash value of the pre_s
    hash_pre_s = hash(pre_s)
    
    # set the parameters
    n = len(lat)
    
    changed_lat = lat.copy()
    
    # randomly choose a subset of the latent with gamma percent
    green_subset = []
    index = []
    for i in range(n):
        if random_number(hash_pre_s + i) % 100 < gamma * 100:
            green_subset.append(changed_lat[i])
            index.append(i)
    test_green_subset = green_subset.copy()
    softmax_list = softmax(green_subset)
    # renew the green subset
    for i in range(len(green_subset)):
        softmax_value = softmax_list[i]  # Compute softmax for the current value
        lambda_param = max(beta * softmax_value, delta * max(lat))  # Calculate lambda for noise
        noise = np.random.normal(lambda_param, max(softmax_list) ) 
        green_subset[i] += noise  # Add Poisson noise
        green_subset[i] = max(0, green_subset[i]) # Clip to zero
    
    # output the result after the process
    for i in range(len(index)):
        value = green_subset[i]
        changed_lat[index[i]] = value
    
    return lat, changed_lat, test_green_subset, index

In [5]:
def test_loop (lat_fun = 10 * np.random.rand(), length = 1000, gamma = 0.5, beta = 7, delta = 0.1, print_info = False):
    num = 0
    change_num = 0
    
    for _ in range(length):
        # set the seed
        seed = time.time()
        np.random.seed(int(seed*1000)%1000)
        lat_test = lat_fun(1000)
        # get the latent using a set of random number
        latent = lat_test.copy()
        
        # set the parameters
        gamma = gamma
        beta = beta
        delta = delta
        
        #  set a random the pre_s
        pre_s = np.random.rand()
        
        # get the result
        lat, ch_lat, green_set, indeces = P_WM(latent, gamma, beta, delta, pre_s)
        result, result_WM = softmax(lat), softmax(ch_lat)
        
        # plot the result
        #plt.plot(result, label = 'Original')
        #plt.plot(result_WM, label = 'With Mechanism')
        #plt.legend()
        #plt.show()
        
        #plt.plot(result_WM - result)
        #plt.show()
        
        max_in_green = indeces[ np.argmax(green_set)]
        new_max_index = np.argmax(result_WM)
        org_max_index = np.argmax(result)
        
        if org_max_index == new_max_index:
            change_num += 1
        
        if (max_in_green != new_max_index and max_in_green != org_max_index and org_max_index == new_max_index and lat[new_max_index] - lat[max_in_green] < 0.5*lat[new_max_index]) or (max_in_green != new_max_index and max_in_green != org_max_index and org_max_index != new_max_index and lat[new_max_index] - lat[max_in_green] > 0.1 * lat[max_in_green]) or (org_max_index != new_max_index and lat[org_max_index] - lat[new_max_index] > 0.5 * lat[org_max_index]):
            if print_info == True:
                print("max_in_green: ", max_in_green)
                print("new: ", new_max_index, " org: ", org_max_index)
                print("max_in_green value, new: ", result_WM[max_in_green], " org: ", result[max_in_green])
                print("WM new: ", result_WM[new_max_index], " org: ", result[new_max_index])
                print("ORG new: ", result_WM[org_max_index], " org: ", result[org_max_index])
            num += 1
        
    return num, change_num
            

In [6]:
def generate_latent_uniform(size=1000):
    """生成一个均匀分布的随机数组。"""
    return 10 * np.random.rand(size)

def generate_latent_normal(size=1000, mean=10, std=2):
    """生成一个正态分布的随机数组。"""
    return np.random.normal(loc=mean, scale=std, size=size)

In [ ]:
test_loop(lat_fun = generate_latent_uniform, length = 10000, gamma = 0.5, beta = 5, delta = 0.1, print_info=True)

0


In [ ]:
# lat_test is following the normal distribution
test_loop(lat_fun =  generate_latent_normal, length = 10000, gamma = 0.5, beta = 5, delta = 0.1, print_info=True)

max_in_green:  560
new:  589  org:  589
max_in_green value, new:  0.03319575583897799  org:  0.017723152171426934
WM new:  0.040259691576378695  org:  0.10997297818230527
ORG new:  0.040259691576378695  org:  0.10997297818230527
max_in_green:  835
new:  853  org:  853
max_in_green value, new:  0.10435535584509968  org:  0.04145890224192814
WM new:  0.105578429167347  org:  0.30026131248356286
ORG new:  0.105578429167347  org:  0.30026131248356286
max_in_green:  11
new:  937  org:  937
max_in_green value, new:  0.05671435300903735  org:  0.022344581875002916
WM new:  0.10245210671902051  org:  0.2353904810210695
ORG new:  0.10245210671902051  org:  0.2353904810210695
max_in_green:  283
new:  789  org:  789
max_in_green value, new:  0.05504319741981903  org:  0.02355023809803341
WM new:  0.09551034876079556  org:  0.25027454440518565
ORG new:  0.09551034876079556  org:  0.25027454440518565
max_in_green:  824
new:  931  org:  931
max_in_green value, new:  0.03827978335614665  org:  0.0203

In [ ]:
test_loop(lat_fun =  generate_latent_normal, length = 10000, gamma = 0.5, beta = 5, delta = 0.1)

518
5336


In [ ]:
test_loop(lat_fun =  generate_latent_normal, length = 100000, gamma = 0.5, beta = 5, delta = 0.1)

5473
54142


In [1357]:
num_list = []
change_num_list = []

for _ in range(100):
    num_ele, change_num_ele = test_loop(lat_fun =  generate_latent_normal, length = 1000, gamma = 0.5, beta = 5, delta = 0.1)
    num_list.append(num_ele)
    change_num_list.append(change_num_ele)
    
print("The number of the non-good: ", np.mean(num_list))
print("The number of the non-change: ", np.mean(change_num_list))
print("The variance of the non-change: ", np.var(change_num_list))


The number of the non-good:  53.57
The number of the non-change:  538.44
The variance of the non-change:  157.30640000000002


In [14]:
setting = True
if setting == True:
    num_list = []
    change_num_list = []
    
    for _ in range(1000):
        num_ele, change_num_ele = test_loop(lat_fun =  generate_latent_normal, length = 100, gamma = 0.5, beta = 5, delta = 0.1)
        num_list.append(num_ele)
        change_num_list.append(change_num_ele)
        
    print("The number of the non-good: ", np.mean(num_list))
    print("The number of the non-change: ", np.mean(change_num_list))
    print("The variance of the non-change: ", np.var(change_num_list))

The number of the non-good:  5.4
The number of the non-change:  53.237
The variance of the non-change:  19.426831000000004


In [7]:
num_list = []
change_num_list = []

for _ in range(100):
    num_ele, change_num_ele = test_loop(lat_fun = generate_latent_uniform , length = 100, gamma = 0.5, beta = 5, delta = 0.1)
    num_list.append(num_ele)
    change_num_list.append(change_num_ele)
    
print("The number of the non-good: ", np.mean(num_list))
print("The number of the non-change: ", np.mean(change_num_list))
print("The variance of the non-change: ", np.var(change_num_list))

The number of the non-good:  0.0
The number of the non-change:  31.39
The variance of the non-change:  20.037899999999993
